In [30]:
%matplotlib inline
import os

import numpy as np
from matplotlib import pyplot as plt

In [31]:
import matplotlib as mpl
fontsize = 14
rcparams = {}
rcparams['figure.figsize'] = (10, 6)
rcparams['axes.labelsize'] = fontsize
rcparams['legend.fontsize'] = fontsize-4
rcparams['axes.titlesize'] = fontsize
rcparams['axes.linewidth'] = 1.5
#rcparams['axes.grid'] = True
for axis in ['x', 'y']:
    rcparams['%stick.labelsize' % axis] = fontsize-2
    rcparams['%stick.direction' % axis] = 'in'
    rcparams['%stick.major.size' % axis] = 5.5
    rcparams['%stick.minor.size' % axis] =  3.5
    rcparams['%stick.major.width' % axis] = 2
    rcparams['%stick.minor.width' % axis] = 1.5
rcparams['xtick.top'] = True
rcparams['ytick.right'] = True
# set up the plotting params
for key in rcparams: mpl.rcParams[key] = rcparams[key]

In [32]:
# setup the outdir - where plots will be saved
outdir = '/pscratch/sd/u/uendert/test_y1_full_shape/figs/'
os.makedirs(outdir, exist_ok=True)

In [33]:
from desilike.samples import plotting, Chain
from getdist import plots

# desilike
def process_chains(chain_dir=os.path.join(os.getenv('SCRATCH'), 'test_y1_full_shape'),
                   observable='corr', tracer='LRG1',
                   templates=['shapefit', 'shapefit-qisoqap', 'shapefit-qisoqap-prior']
                ):
    # set up zmin, zmax for this tracer
    zmin, zmax = {'LRG0': [0.4, 1.1],'LRG1': [0.4, 0.6], 'LRG2': [0.6, 0.8], 'LRG3': [0.8, 1.1],
                   'ELG1':[0.8, 1.1], 'ELG2':[0.8, 1.6], 'ELG3': [1.1, 1.6],
                   'QSO1': [0.8, 2.1],
                   'BGS1': [0.1, 0.4]
                   
                   }[tracer]

    # modify the names to be what the files have
    if 'ELG' in tracer: tracer = 'ELG_LOPnotqso' + tracer[-1]
    if 'BGS' in tracer: tracer = 'BGS_BRIGHT-21.5' + tracer[-1]
    # misc
    templates = np.array(templates)
    templates_readin = []

    # load in the chains
    chains = []
    # loop over the templates - and read whats available
    for template in templates:
        print(f'reading chains for {template} .. ')
        # pscratch/sd/u/uendert/test_y1_full_shape/chain_LRG_0.4_1.1_power_velocileptors_shapefit-qisoqap_0.npy
        # /pscratch/sd/u/uendert/test_y1_full_shape/chain_LRG_0.4_0.6_power_velocileptors_shapefit_0.npy

        print(os.path.join(chain_dir, 'chain_{}_{}_{}_{}_velocileptors_{}_0.npy'.format(tracer[:-1], zmin, zmax, observable, template,)))
        if os.path.exists(os.path.join(chain_dir, 'chain_{}_{}_{}_{}_velocileptors_{}_0.npy'.format(tracer[:-1], zmin, zmax, observable, template,))):
            chain = Chain.concatenate([Chain.load(os.path.join(chain_dir,
                                                            'chain_{}_{}_{}_{}_velocileptors_{}_{:d}.npy'.format(tracer[:-1], zmin, zmax, observable, template,
                                                                                                                    i))).remove_burnin(0.5)[::20] for i in range(8)])
            chain = chain.sample_solved()
            chains.append(chain.to_getdist(label='template: {}'.format(template)))
            templates_readin.append(template)
        else:
            print(f'chain for {template} not found')

    # now plot - first the full thing, with all params
    #colors = ['#7f7f7f', '#1f77b4', '#ff7f0e']
    colors = ['#7f7f7f', '#1f77b4', '#ff7f0e']
    # ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    plt.clf()
    g = plots.get_subplot_plotter()
    g.settings.alpha_filled_add = 0.8
    g.settings.title_limit_fontsize = 20
    g.settings.axes_labelsize = 20
    g.settings.legend_fontsize = 20
    g.triangle_plot(chains, analysis_settings={'ignore_rows': 0.3},
                    filled=False, contour_lws=[3,2,2],
                    contour_colors=colors,
                    markers={"qper":1., "qpar":1., "df":1., "dm":0.}, marker_args={'ls': '-'},
                    )
    plt.tight_layout()
    plt.suptitle(f"{tracer[:-1]}: [{zmin},{zmax}]", fontweight="bold", y=1.05, fontsize=20)
    fname = 'plot_posteriors_{}_{}_{}_{}_velocileptors_{}templates.png'.format(tracer[:-1], zmin, zmax, observable, len(templates_readin))
    plt.savefig(f'{outdir}/{fname}', format='png', bbox_inches='tight')
    print(f'saved {fname}')
    plt.show()

    # now plot only some
    if 'shapefit-qisoqap' in templates_readin and 'shapefit-qisoqap-prior' in templates_readin:
        ind1 = np.where(templates == 'shapefit-qisoqap')[0][0]
        ind2 = np.where(templates == 'shapefit-qisoqap-prior')[0][0]

        plt.clf()
        g = plots.get_subplot_plotter()
        g.settings.alpha_filled_add = 0.8
        g.settings.title_limit_fontsize = 14
        g.settings.axes_labelsize = 14
        g.settings.legend_fontsize = 14
        g.settings.axes_fontsize = 14
        g.triangle_plot([chains[ind1],chains[ind2]], params=["qiso", "qap", "df", "dm"],
                        analysis_settings={'ignore_rows': 0.3}, title_limit=2,
                        markers={"qiso":1., "qap":1., "df":1., "dm":0.}, marker_args={'ls': '-'},
                        filled=False, contour_lws=[3,2],
                        contour_colors=[colors[ind1], colors[ind2]]
                        )
        plt.tight_layout()
        plt.suptitle(f"{tracer[:-1]}: [{zmin},{zmax}]", fontweight="bold")
        fname = 'plot_posteriors-4params_{}_{}_{}_{}_velocileptors_2templates.png'.format(tracer[:-1], zmin, zmax, observable)
        plt.savefig(f'{outdir}/{fname}', format='png', bbox_inches='tight')
        print(f'saved {fname}')
        plt.show()

In [34]:
%%time

for tracer in ['LRG0', 'LRG1', 'LRG2', 'LRG3', 'ELG1', 'ELG2', 'ELG3', 'QSO1', 'BGS1']:
   print('############################################################')
   print(f'plotting things for {tracer}')
   process_chains(chain_dir=os.path.join(os.getenv('SCRATCH'), 'test_y1_full_shape'),
                   observable='corr', tracer=tracer,
                   templates=['shapefit', 'shapefit-qisoqap', 'shapefit-qisoqap-prior']
                )
   print('############################################################\n\n')

############################################################
plotting things for LRG0
reading chains for shapefit .. 
/pscratch/sd/u/uendert/test_y1_full_shape/chain_LRG_0.4_1.1_corr_velocileptors_shapefit_0.npy
chain for shapefit not found
reading chains for shapefit-qisoqap .. 
/pscratch/sd/u/uendert/test_y1_full_shape/chain_LRG_0.4_1.1_corr_velocileptors_shapefit-qisoqap_0.npy
chain for shapefit-qisoqap not found
reading chains for shapefit-qisoqap-prior .. 
/pscratch/sd/u/uendert/test_y1_full_shape/chain_LRG_0.4_1.1_corr_velocileptors_shapefit-qisoqap-prior_0.npy
chain for shapefit-qisoqap-prior not found


IndexError: list index out of range

<Figure size 1000x600 with 0 Axes>

misc

In [ ]:

#g = plots.get_subplot_plotter()

#?g.settings
#?g.triangle_plot

In [ ]:
[f for f in os.listdir('/pscratch/sd/u/uendert/test_y1_full_shape/') if f.__contains__('ELG') and f.__contains__('0.8_1.1')]

['chain_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit-qisoqap_4.npy',
 'chain_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit-qisoqap-prior_5.npy',
 'chain_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit-qisoqap_1.npy',
 'chain_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit-qisoqap_6.npy',
 'chain_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit_7.npy',
 'chain_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit-qisoqap-prior_0.npy',
 'chain_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit-qisoqap-prior_7.npy',
 'chain_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit-qisoqap-prior_6.npy',
 'chain_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit-qisoqap_7.npy',
 'chain_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit_6.npy',
 'chain_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit_1.npy',
 'poles_bestfit_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit-qisoqap.png',
 'chain_ELG_LOPnotqso_0.8_1.1_corr_velocileptors_shapefit_2.npy',
 'chain_ELG_LOPnotqso_0.8_1.1_corr_veloc

In [ ]:
temp = chains[1]
-2 * temp.loglikes.max()

NameError: name 'chains' is not defined

In [ ]:
from getdist import plots
plt.clf()
g = plots.get_subplot_plotter()
g.triangle_plot(chains, params=['LRG.b1', 'LRG.b2', 'LRG.bs', 'LRG.alpha0', 'LRG.alpha2', 'LRG.alpha4', 'LRG.sn2', 'LRG.sn2','LRG.sn4'], analysis_settings={'ignore_rows': 0.3})
plt.tight_layout()
plt.suptitle(f"{tracer[:-1]}: [{zmin},{zmax}]", fontweight="bold")
plt.show()

In [ ]:
for i in range(12):
    print(i, chains[0].parName(i))

In [ ]:
print(chains[1].getTable().tableTex())